# hybrid Search

In [14]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from dotenv import load_dotenv

In [15]:
import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("openai api 키가 없습니다. 한번더 확인 부탁드립니다.")

os.environ['OPENAI_API_KEY'] = openai_api_key

In [16]:
# Loader
loader = PyPDFLoader("sample.pdf")
pages = loader.load_and_split()
pages

[Document(metadata={'source': '2024-24-03 12:24:19-save.pdf', 'page': 0}, page_content='에코프로 공급물량  신청  사이트 ?…" 사\n기입니다""60세 이상 고용률  높이면  잠재성장률\n증가"…노동시장  유연성  전제돼야\n\'오염물질· 온실가스  무배출 \' 수소차  첫\n등록 5년 만에  1000 대  돌파다시 불어온  청약  광풍… 하루에  17 만명\n몰려\n롯데바이오, 송도에  바이오의약품  생산\n플랜트 착공… 경제효과  7.6 조원대기업 CEO 들도  \' 고령화 \'… 서울대  출\n신은 줄었다\n“엄청난 회복세”… 테슬라, 주가  10%\n넘게 급등이복현 금감원장  " 증권사, 손쉬운  수익\n원 찾는 영업  관행  바뀌어야 "\n2027년부터  초소형  관측위성이  한반\n도 온실가스  감시한다카카오뱅크, 대구신보와  375 억  규모\n소상공인 지원  협약헤드라인 뉴스  \n연합뉴스\n16\n 데일리안\n12\n뉴스1\n11\n 한국경제TV\n32\n헤럴드경제\n43\n 파이낸셜뉴스\n15\n한겨레\n35\n SBS\n35\n경향신문\n32\n 머니투데이\n12\n원초적 질문\n상시근로자 4.9 명은  5 명  미만일까  이상일까\n더스쿠프\n금투세 시행하면  정말  \' 부양가족  인적공제 \' 못  받아 ?더스쿠프100+\n"본사 영업이익률  41.2%" 컴포즈커피  가맹점도  좋을까더스쿠프10+\n종부세는 정말  \' 불편한  세금 \' 일까더스쿠프100+\n2,791.90\n11.04+0.40%\n832.64\n2.73+0.33%\n금융 증권 부동산언론사별정치경제사회생활/문화IT/과학세계랭킹신문보기오피니언TV팩트체크알고리즘'),
 Document(metadata={'source': '2024-24-03 12:24:19-save.pdf', 'page': 1}, page_content='윤석열 "배당하면  세제혜택 "..." 배당소\n득세, 저율분리과세" [ 오한마 

In [4]:
# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

In [5]:
texts = text_splitter.split_documents(pages)
texts

[Document(metadata={'source': 'sample.pdf', 'page': 0}, page_content='에코프로 공급물량  신청  사이트 ?…" 사\n기입니다""60세 이상 고용률  높이면  잠재성장률\n증가"…노동시장  유연성  전제돼야\n\'오염물질· 온실가스  무배출 \' 수소차  첫\n등록 5년 만에  1000 대  돌파다시 불어온  청약  광풍… 하루에  17 만명\n몰려'),
 Document(metadata={'source': 'sample.pdf', 'page': 0}, page_content='몰려\n롯데바이오, 송도에  바이오의약품  생산\n플랜트 착공… 경제효과  7.6 조원대기업 CEO 들도  \' 고령화 \'… 서울대  출\n신은 줄었다\n“엄청난 회복세”… 테슬라, 주가  10%\n넘게 급등이복현 금감원장  " 증권사, 손쉬운  수익\n원 찾는 영업  관행  바뀌어야 "'),
 Document(metadata={'source': 'sample.pdf', 'page': 0}, page_content='원 찾는 영업  관행  바뀌어야 "\n2027년부터  초소형  관측위성이  한반\n도 온실가스  감시한다카카오뱅크, 대구신보와  375 억  규모\n소상공인 지원  협약헤드라인 뉴스  \n연합뉴스\n16\n 데일리안\n12\n뉴스1\n11\n 한국경제TV\n32\n헤럴드경제\n43\n 파이낸셜뉴스\n15'),
 Document(metadata={'source': 'sample.pdf', 'page': 0}, page_content="32\n헤럴드경제\n43\n 파이낸셜뉴스\n15\n한겨레\n35\n SBS\n35\n경향신문\n32\n 머니투데이\n12\n원초적 질문\n상시근로자 4.9 명은  5 명  미만일까  이상일까\n더스쿠프\n금투세 시행하면  정말  ' 부양가족  인적공제 ' 못  받아 ?더스쿠프100+"),
 Document(metadata={'source': 'sample.pdf', 'page': 0}, pa

In [6]:
# Embedding
embeddings_model = OpenAIEmbeddings()

In [7]:
# load it into Chroma
vectorstore = Chroma.from_documents(texts, embeddings_model)

chroma_retriever = vectorstore.as_retriever(
    search_type="mmr", # MMR 알고리즘을 사용하여 검색
    search_kwargs={'k':1,'fetch_k':4} # 상위 1개의 문서를 반환하지만, 고려할 문서는 4개로 설정
)

### pip install rank_bm25

In [8]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
# Initialize the BM25 retriever
bm25_retriever = BM25Retriever.from_documents(texts)
bm25_retriever.k = 2  # Retrieve top 2 results

print("type of bm25", type(bm25_retriever))

type of bm25 <class 'langchain_community.retrievers.bm25.BM25Retriever'>


# Initialize the ensemble retriever


In [9]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, chroma_retriever], weights=[0.2, 0.8]
)

In [12]:
# 예시 고객 문의
query = "에코 프로에 대해서 알려줘"

# 관련 문서/제품 검색
docs = ensemble_retriever.get_relevant_documents(query)

# 각 문서에서 페이지 내용을 추출하여 출력
# for doc in docs:
#     print(doc.page_content)

# 검색된 문서들 출력
docs

에코프로 공급물량  신청  사이트 ?…" 사
기입니다"
33분전영풍, 고려아연  황산  취급  중단에  법적
대응
34분전
한국경제TV
 한겨레
뉴스1
 한국일보
문화일보
 데일리안
오마이뉴스
 SBS Biz
문화일보
 문화일보
SBS Biz
 더팩트
이 콘텐츠의  저작권은  저작권자  또는  제공처에  있으며 , 이를  무단  이용하는 경우  저작권법  등에  따라  법적  책임을  질  수  있습니다 .
논의 시작할까
32분전'진격의 채권개미' 올  들어  5 조  싹쓸
이…편법 영업도  기승
33분전
한화시스템, 405 억  초소형  온실가스
관측 위성 사업  수주
33분전분양 가뭄… 과천· 마포에  청약통장  17
만개 ‘우르르ʼ
33분전
에코프로 공급물량  신청  사이트 ?…" 사


In [11]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

template = """다음 맥락을 바탕으로 질문에 답변하세요:

{context}

질문: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# LLM
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Post-processing
def format_docs(docs):
    formatted = "\n\n".join(doc.page_content for doc in docs)
    return formatted

# Chain
rag_chain = (
    {"context": ensemble_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
rag_chain.invoke("에코프로에 대해서 알려줘")

'에코프로는 주로 환경 관련 기술 및 제품을 개발하고 공급하는 기업으로, 특히 전지 및 배터리 관련 사업에 주력하고 있습니다. 이 회사는 전기차 및 에너지 저장 시스템에 사용되는 리튬 이온 배터리의 핵심 소재인 양극재를 생산하는 데 중요한 역할을 하고 있습니다. 에코프로는 지속 가능한 발전과 친환경 기술을 강조하며, 다양한 환경 문제 해결을 위한 혁신적인 솔루션을 제공하고 있습니다. 최근에는 공급물량 신청과 관련된 이슈가 언급되었으며, 이는 에코프로의 제품 수급과 관련된 중요한 사항일 수 있습니다.'